In [1]:
# [Cell: Physics Engine]
import torch
import torch.nn as nn
try:
    from torch_scatter import scatter_add
except ImportError:
    print("❌ Error: torch_scatter not installed. Please install it for efficient physics computation.")
    # Fallback for CPU-only or simple testing (slower)
    def scatter_add(src, index, dim=-1, out=None, dim_size=None):
        return out.scatter_add_(dim, index, src)

class PowerFlowLoss(nn.Module):
    """
    ⚡ 可微潮流物理层 (Differentiable Power Flow Layer)
    利用基尔霍夫定律计算功率不平衡量 (Power Mismatch)。
    完全支持 Batch 并行计算。
    """
    def __init__(self, device):
        super().__init__()
        self.device = device
        
    def forward(self, pred_v_mag, pred_v_ang, edge_index, edge_attr_real, batch_idx):
        """
        输入:
            pred_v_mag: [N_total, 1] 预测电压幅值 (p.u.)
            pred_v_ang: [N_total, 1] 预测电压相角 (radians)
            edge_index: [2, E_total] 拓扑连接
            edge_attr_real: [E_total, 2] 真实阻抗 (R, X)
            batch_idx: [N_total] 节点所属的 Batch ID (用于统计每个图的 Loss)
            
        输出:
            P_calc, Q_calc: [N_total, 1] 计算得到的节点注入功率
        """
        # 1. 准备导纳 (Admittance)
        # Y = 1 / (R + jX)
        r = edge_attr_real[:, 0]
        x = edge_attr_real[:, 1]
        z = torch.complex(r, x)
        y_branch = 1 / (z + 1e-6) # [E_total]
        
        # 2. 构建复数电压 V
        # V = |V| * e^(j*theta)
        v_complex = torch.polar(pred_v_mag.squeeze(), pred_v_ang.squeeze()) # [N_total]
        
        # 3. 计算支路电流 (Branch Current)
        # I_ij = (V_i - V_j) * Y_ij
        src, dst = edge_index
        v_i = v_complex[src]
        v_j = v_complex[dst]
        
        # 广播 y_branch
        i_branch = (v_i - v_j) * y_branch # [E_total]
        
        # 4. 聚合节点注入电流 (Nodal Injection Current)
        # I_i = sum_{j} I_ij (流出为正)
        # 注意：edge_index 是双向的，所以对每个节点 i，所有以 i 为 src 的边都代表流出
        # 我们需要初始化一个全零张量
        i_inj = torch.zeros_like(v_complex)
        
        # 使用 scatter_add 将同一 src 的电流加起来
        # dim=0 因为 i_branch 是 1D 的 (或者 [E])
        scatter_add(i_branch, src, dim=0, out=i_inj) # [N_total]
        
        # 5. 计算复数功率 S_inj = V_i * conj(I_inj)
        # 注意：这里计算的是注入功率 (Generation - Load)
        s_calc = v_complex * torch.conj(i_inj) # [N_total]
        
        p_calc = s_calc.real.unsqueeze(-1) # [N_total, 1]
        q_calc = s_calc.imag.unsqueeze(-1) # [N_total, 1]
        
        return p_calc, q_calc

print("✅ 物理引擎 (PowerFlowLoss) 定义完成。")

OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\10856\miniconda3\envs\lvdn-diff\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.